### dependencies & create nationwide FIPS dictionary


In [1]:
import pandas as pd
import requests
from datetime import timedelta, datetime
import plotly.express as px

# Gloabal variables
state_fips = '47'
metro_name = 'Nashville'
wiki_link = 'https://en.wikipedia.org/wiki/List_of_counties_in_Tennessee'
starting_year = 2022
county_dict = {
    '47015': 'Cannon',
    '47021': 'Cheatham',
    '47037': 'Davidson',
    '47043': 'Dickson',
    '47081': 'Hickman',
    '47111': 'Macon',
    '47119': 'Maury',
    '47147': 'Robertson',
    '47149': 'Rutherford',
    '47159': 'Smith',
    '47165': 'Sumner',
    '47169': 'Trousdale',
    '47187': 'Williamson',
    '47189': 'Wilson'
}

fips_dict = {
    '01': 'Alabama',
    '02': 'Alaska',
    '04': 'Arizona',
    '05': 'Arkansas',
    '06': 'California',
    '08': 'Colorado',
    '09': 'Connecticut',
    '10': 'Delaware',
    '11': 'District of Columbia',
    '12': 'Florida',
    '13': 'Georgia',
    '15': 'Hawaii',
    '16': 'Idaho',
    '17': 'Illinois',
    '18': 'Indiana',
    '19': 'Iowa',
    '20': 'Kansas',
    '21': 'Kentucky',
    '22': 'Louisiana',
    '23': 'Maine',
    '24': 'Maryland',
    '25': 'Massachusetts',
    '26': 'Michigan',
    '27': 'Minnesota',
    '28': 'Mississippi',
    '29': 'Missouri',
    '30': 'Montana',
    '31': 'Nebraska',
    '32': 'Nevada',
    '33': 'New Hampshire',
    '34': 'New Jersey',
    '35': 'New Mexico',
    '36': 'New York',
    '37': 'North Carolina',
    '38': 'North Dakota',
    '39': 'Ohio',
    '40': 'Oklahoma',
    '41': 'Oregon',
    '42': 'Pennsylvania',
    '44': 'Rhode Island',
    '45': 'South Carolina',
    '46': 'South Dakota',
    '47': 'Tennessee',
    '48': 'Texas',
    '49': 'Utah',
    '50': 'Vermont',
    '51': 'Virginia',
    '53': 'Washington',
    '54': 'West Virginia',
    '55': 'Wisconsin',
    '56': 'Wyoming'
}

df_FIPS = pd.read_csv('nationwide_fips_lookup.csv', dtype={'fips_code': 'str'})
df_Tennessee = df_FIPS[df_FIPS['fips_code'].str.startswith(state_fips)]

df_Tennessee

,fips_code,county_name
2427,47001,"Anderson County, TN"
2428,47003,"Bedford County, TN"
2429,47005,"Benton County, TN"
2430,47007,"Bledsoe County, TN"
2431,47009,"Blount County, TN"
...,...,...
2517,47181,"Wayne County, TN"
2518,47183,"Weakley County, TN"
2519,47185,"White County, TN"
2520,47187,"Williamson County, TN"


### now get population, merge with above table


In [2]:
# Read all tables from the page
county_Poptable = pd.read_html(wiki_link, header=0)[1]

county_Poptable = county_Poptable.rename(columns={
    'FIPS code[8]': 'FIPS',
    'Population': 'population'
})

county_Poptable = county_Poptable[[
    'County',
    'FIPS',
    'population'
]]

# create the full FIPS code
county_Poptable['FIPS'] = '47' + \
    county_Poptable['FIPS'].astype(str).str.zfill(3)

df_Tennessee_pop = pd.merge(
    df_Tennessee,
    county_Poptable,
    how='left',
    left_on='fips_code',
    right_on='FIPS'
).drop(columns=['County', 'fips_code'])

# remove the " County" substring from 'County' column
df_Tennessee_pop['county_name'] = df_Tennessee_pop['county_name'].str.replace(
    ' County, TN', '')

df_Tennessee_pop

,county_name,FIPS,population
0,Anderson,47001,80234
1,Bedford,47003,53055
2,Benton,47005,16103
3,Bledsoe,47007,15060
4,Blount,47009,141456
...,...,...,...
90,Wayne,47181,16066
91,Weakley,47183,33084
92,White,47185,28692
93,Williamson,47187,264460


### To make this work, you have to quickly grab the URL that opens when you

### elect to download SOCDS data as a CSV


In [3]:
# dynamically construct the query parameters for years & counties
current_year = datetime.now().year
year_list = list(range(starting_year, current_year + 1))
year_list_str = '%23'.join(map(str, year_list))
county_list_str = '%23'.join(map(str, county_dict.keys()))


# read in the data; have to get this URL that is generated when you click to download the SOCDS data as a CSV
url = f"https://socds.huduser.gov/permits/output_monthly_csv.odb?outpref=csv&geoval=state&datatype=monthlyP&varlist=1%232%233%234%235%236&yearlist={year_list_str}&statelist=13&msalist=+&cbsalist=+&bppllist=+&cntylist={county_list_str}&COUNTYSUM=YES&COUNTYALL=+&COUNTYGRP=+&STATESUM=+&STATEALL=+&METROSUM=+&METROALL=+&METRO=+&CBSA=+&PLACEGRP=+&CSUMNAME=&JSUMNAME=+&geo=state&chron=monthlyP&_ga=GA1.1.1317611679.1717085682&_ga_YD74G0644V=GS1.1.1717549416.5.0.1717549416.0.0.0&_ga_CSLL4ZEK4L=GS1.1.1717549416.5.0.1717549416.0.0.0&__utmc=200893484&__utma=200893484.1317611679.1717085682.1719969523.1720011864.3&__utmz=200893484.1720011864.3.3.utmcsr%3Dgoogle%7Cutmccn%3D%28organic%29%7Cutmcmd%3Dorganic%7Cutmctr%3D%28not+provided%29&__utmt=1&__utmb=200893484.1.10.1720011864"
df = pd.read_csv(url)

# extract just the county name
df['county_name'] = df['Location'].str.split(' County').str[0]

# Create a date column
df['date'] = pd.to_datetime(df[['Year', 'Month']].assign(day=1))

# rename values in the 'Series' column
df['Series'] = df['Series'].replace({
    'Units in Single-Family Structures': 'Single-Family Permits',
    'Units in All Multi-Family Structures': 'Multi-Family Permits',
    'Units in 2-unit Multi-Family Structures': 'Multi-Family Units (2 units)',
    'Units in 3- and 4-unit Multi-Family Structures': 'Multi-Family Units (3-4 units)',
    'Units in 5+ Unit Multi-Family Structures': 'Multi-Family Units (5+ units)'
})

# remove the " County" from "Location"
df['Location'] = df['Location'].str.replace(' County', '')

df = pd.merge(
    df,
    df_Tennessee_pop,
    how='left',
    left_on='Location',
    right_on='county_name'
).drop(columns='county_name_x').rename(columns={'county_name_y': 'county_name'})

# pare down columns
df = df[[
    'county_name',
    'FIPS',
    'population',
    'date',
    'Series',
    'Permits'
]]

# Create 'month_year' column with abbreviated month and year
df['month_year'] = df['date'].dt.strftime('%b %Y')

# After some consideration, and for Larry Long, just grab 'Single-Family Units';
# but....available series include: 'All Permits', 'Single-Family Units', 'All Multifamily',
# '2-Unit Multifamily', '3 & 4-Unit Multifamily', & '5+-Unit Multifamily'
df = df[df['Series'] == 'Single-Family Permits']
df = df.drop(columns='Series')

# to see where the data 'stops', we need to group on the month_year first
monthly_permits = df.groupby('month_year')['Permits'].sum().reset_index()

# select the 'month_year' values that don't have any permit data and remove
months_with_0_permits = monthly_permits[
    monthly_permits['Permits'] == 0]['month_year']

# remove these 0-permit months from consideration
df = df[~df['month_year'].isin(months_with_0_permits)]

df.tail(8)

,county_name,FIPS,population,date,Permits,month_year
2965,Wilson,47189,163674,2024-02-01,163.0,Feb 2024
2966,Wilson,47189,163674,2024-03-01,76.0,Mar 2024
2967,Wilson,47189,163674,2024-04-01,129.0,Apr 2024
2968,Wilson,47189,163674,2024-05-01,203.0,May 2024
2969,Wilson,47189,163674,2024-06-01,210.0,Jun 2024
2970,Wilson,47189,163674,2024-07-01,227.0,Jul 2024
2971,Wilson,47189,163674,2024-08-01,230.0,Aug 2024
2972,Wilson,47189,163674,2024-09-01,154.0,Sep 2024


### Create the KPI dataframe which will show 12-month totals


In [4]:
# Filter dataframe to include only the last 12 months
max_date = df['date'].max()
min_date = max_date - timedelta(days=365)
df_filtered = df[(df['date'] >= min_date) & (df['date'] <= max_date)]

# Group by county_name and aggregate permits to get 12-month totals
grouped = df_filtered.groupby('county_name').agg(
    total_permits=('Permits', 'sum'),
    population=('population', 'first')
).reset_index()

grouped['permit_ratio'] = (
    grouped['total_permits'] / grouped['population']) * 10000
print(
    f'metro permit ratio: {(grouped["total_permits"].sum() / grouped["population"].sum())*10000}')
grouped.to_csv('building_permits_KPI.csv', index=False)
print('export complete!')
grouped

metro permit ratio: 70.20445901283809
export complete!


,county_name,total_permits,population,permit_ratio
0,Cannon,163.0,15063,108.212176
1,Cheatham,284.0,42254,67.212572
2,Davidson,2746.0,712334,38.549332
3,Dickson,378.0,56729,66.632587
4,Hickman,122.0,25826,47.239216
5,Macon,152.0,26793,56.731236
6,Maury,1647.0,110760,148.699892
7,Robertson,958.0,76776,124.778577
8,Rutherford,2982.0,367101,81.231051
9,Smith,129.0,20538,62.810400


### Create metrowide monthly totals which will power the line chart


In [5]:
# Calculate the total population for 'metro'
unique_populations = df_filtered[[
    'county_name', 'population']].drop_duplicates()
total_population = unique_populations['population'].sum()

# Aggregate permits for each month
metro_data = df_filtered.groupby('date').agg({'Permits': 'sum'}).reset_index()
metro_data['county_name'] = 'Metro'
metro_data['FIPS'] = 'n/a'
metro_data['month_year'] = metro_data['date'].dt.strftime('%b %Y')
metro_data['population'] = total_population

# Reorder columns to match the original DataFrame
metro_data = metro_data[[
    'county_name',
    'FIPS',
    'population',
    'date',
    'Permits',
    'month_year',
]]

# Concatenate the metrowide with the original, filtered data
df_final = pd.concat([df_filtered, metro_data], ignore_index=True)

# create the permit ratio
df_final['permit_ratio'] = (
    df_final['Permits'] / df_final['population']) * 10000

df_final.to_csv('building_permits.csv', index=False)
print('export complete!')
df_final.head(3)

export complete!


,county_name,FIPS,population,date,Permits,month_year,permit_ratio
0,Cannon,47015,15063,2023-10-01,12.0,Oct 2023,7.966541
1,Cannon,47015,15063,2023-11-01,8.0,Nov 2023,5.311027
2,Cannon,47015,15063,2023-12-01,8.0,Dec 2023,5.311027
